In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import re

CHEMIN = "C:/Users/mathy/OneDrive/Bureau/EthicsFinance" # A changer selon l'endroit où vous clonez le repo git
esg_grade = pd.read_csv(f'{CHEMIN}/Score ESG brut.csv')

In [ ]:
esg_grade = esg_grade.dropna(subset=['ISSUER_EQUITY_TICKER', 'Groupe Industrie GICS', 'Score ESG brut'])
esg_grade = esg_grade.drop(columns=['Pays', 'Date de Scoring', 'Identifiant Bloomberg Company', 'Identifiant sous industrie GICS', "Nom de l'émetteur"])
esg_grade["ISSUER_EQUITY_TICKER"] = esg_grade["ISSUER_EQUITY_TICKER"].str.strip().str.replace(r'US$', '', regex=True)

In [ ]:
esg_grade = esg_grade[~esg_grade["ISSUER_EQUITY_TICKER"].str.contains(r'\d')]

In [ ]:
top_60_percent = esg_grade.sort_values("Score ESG brut", ascending=False)
top_60_percent = top_60_percent.head(int(0.6 * len(esg_grade)))


In [ ]:
esg_grade["ISSUER_EQUITY_TICKER"] = esg_grade["ISSUER_EQUITY_TICKER"].str.split().str[0]

In [ ]:

end_date = datetime.today().date()
start_date = end_date - timedelta(days=3*365)

historical_data = {}

for ticker in esg_grade["ISSUER_EQUITY_TICKER"].unique():
    try:
        # Essayer de récupérer les données
        data = yf.download(ticker, start=start_date, end=end_date, interval="1d")
        if data.empty:
            print(f"Aucune donnée pour le ticker {ticker}.")
            continue
        data["Ticker"] = ticker
        historical_data[ticker] = data
    except Exception as e:
        print(f"Erreur pour le ticker {ticker} : {e}")

df_all_prices = pd.concat(historical_data.values())


df_all_prices = df_all_prices.reset_index()

df_all_prices.to_csv(f'{CHEMIN}/price_ticker.csv')
print(df_all_prices.head())


In [26]:
fichier_final = pd.read_csv('C:/Users/mathy/OneDrive/Bureau/EthicsFinance/price_ticker.csv')

In [27]:
fichier_final = fichier_final.dropna(axis=1)


In [31]:
fichier_final = fichier_final.apply(pd.to_numeric, errors='coerce')
daily_returns = fichier_final.pct_change()

# Calcul du rendement moyen pour chaque action
mean_returns = daily_returns.mean()

C:\Users\mathy\AppData\Local\Temp\ipykernel_16084\549063680.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = fichier_final.pct_change()


In [32]:
sorted_returns = mean_returns.sort_values(ascending=False)

top_n = int(len(sorted_returns) * 0.5)

# Garder les 50% du haut
top_50_percent = sorted_returns.head(top_n)

In [34]:
top_ticker_names = top_50_percent.index.tolist()

In [36]:
top_ticker_names = pd.DataFrame(top_ticker_names)

In [37]:
top_ticker_names.to_csv(f'{CHEMIN}/tickers_finaux.csv')